In [2]:
%load_ext autoreload
%autoreload 2

import fd_imputer
import pandas as pd
import numpy as np
from sklearn.metrics import f1_score
import itertools

## Set up all paths and labels needed in this notebook

In [2]:
DATA_PATH = 'MLFD_fd_detection/backend/WEB-INF/classes/inputData/adult.csv'
SPLITS_PATH = 'MLFD_fd_detection/data/'
METANOME_DATA_PATH = 'MLFD_fd_detection/backend/WEB-INF/classes/inputData/'
FD_PATH = 'MLFD_fd_detection/results/HyFD-1.2-SNAPSHOT.jar2019-05-07T082200_fds'
DATA_TITLE = 'adult'

## 1. How to split the dataset

In [8]:
df = pd.read_csv(DATA_PATH, sep=';', header=None)
fd_imputer.save_df_split(DATA_TITLE, df, SPLITS_PATH, METANOME_DATA_PATH, [0.8, 0.2])

Dataset successfully written to MLFD_fd_detection/data/adult
Train set successfully written to MLFD_fd_detection/data/train/adult_train
Copied successfully train-dataset to MLFD_fd_detection/backend/WEB-INF/classes/inputData/
Test set successfully written to MLFD_fd_detection/data/test/adult_test


## 2. How to use the ML-imputer

In [6]:
df_train = pd.read_csv(SPLITS_PATH+'test/'+DATA_TITLE+'_test.csv', header=None)
df_test = pd.read_csv(SPLITS_PATH+'train/'+DATA_TITLE+'_train.csv', header=None)
fds = fd_imputer.read_fds(FD_PATH)
impute_column = str(9)
# print(fds)

In [4]:
df_test = df_test.replace('noValueSetHere123156456', np.nan)
df_train = df_train.replace('noValueSetHere123156456', np.nan)

Impute test-set

In [78]:
df_imputed = fd_imputer.ml_imputer(df_train, df_test, impute_column)

2019-05-23 17:40:26,558 [INFO]  CategoricalEncoder for column 9                                found only 63 occurrences of value Amer-Indian-Eskimo
2019-05-23 17:40:26,571 [INFO]  CategoricalEncoder for column 9                                found only 44 occurrences of value Other
2019-05-23 17:40:28,108 [INFO]  
========== start: fit model
2019-05-23 17:40:28,109 [WARNING]  Already bound, ignoring bind()
2019-05-23 17:40:30,164 [INFO]  Epoch[0] Batch [0-184]	Speed: 1460.93 samples/sec	cross-entropy=0.451956	9-accuracy=0.847973
2019-05-23 17:40:32,124 [INFO]  Epoch[0] Train-cross-entropy=0.331838
2019-05-23 17:40:32,125 [INFO]  Epoch[0] Train-9-accuracy=0.885729
2019-05-23 17:40:32,126 [INFO]  Epoch[0] Time cost=4.014
2019-05-23 17:40:32,133 [INFO]  Saved checkpoint to "imputer_model/model-0000.params"
2019-05-23 17:40:32,478 [INFO]  Epoch[0] Validation-cross-entropy=0.157488
2019-05-23 17:40:32,479 [INFO]  Epoch[0] Validation-9-accuracy=0.971037
2019-05-23 17:40:34,559 [INFO]  Epoc

KeyboardInterrupt: 

In [7]:
df_imputed.head()

NameError: name 'df_imputed' is not defined

In [ ]:
f1 = f1_score(df_imputed[impute_column], df_imputed[impute_column+'_imputed'].astype(int), average='weighted')
print(f1)

The ML model's mean relative error is roughly 0.2%

In [ ]:
average_rel_error = ((df_imputed[impute_column] - df_imputed[impute_column+'_imputed'])
 / df_imputed[impute_column]).mean()
print(average_rel_error)

## FD_Imputer

In [9]:
fd = {9: [3, 4, 7, 8]}
list(fd)
fd[list(fd)[0]]

[3, 4, 7, 8]

In [5]:
    df_test = pd.DataFrame([['a', 'b', 'c'], ['d', 'e', 'f'], ['g', 'h', 'i']],
                           columns=[1, 2, 3])
    df_train = pd.DataFrame([['d', 'e', 'f'], ['g', 'h', 'i'], ['a', 'b', 'c']],
                            columns=[1, 2, 3])
    fd = {3: [2, 1]}
    lhs = [2, 1]

In [11]:
df_test 

,1,2,3
0,a,b,c
1,d,e,f
2,g,h,i


In [12]:
df_train

,1,2,3
0,d,e,f
1,g,h,i
2,a,b,c


In [17]:
fd_imputer.select_LHS_row(df_train.iloc[1], df_test, lhs).index[0]

2

In [15]:
df_test.iloc[[1]]

,1,2,3
1,d,e,f


In [10]:
impute_row = df_train.iloc[0]
lhs = fd[list(fd)[0]]

Trivial example: Return the row in df_train where the first-row entry of df_train has a valid LHS counterpart. --> That's at least the first-row entry itself!

In [11]:
fd_imputer.select_LHS_row(df_train.iloc[0], df_train, lhs)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,22855,23,Private,113309,Some-college,10,Never-married,Transport-moving,Not-in-family,White,Male,0,0,40,United-States,<=50K


Now do the same thing for all rows in df_test

In [13]:
for index, row in df_test.iterrows():
    fd_imputer.select_LHS_row(row, df_train, lhs, print_output=True)

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty Data

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty Data

Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty DataFrame
Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
Index: []
Empty Data

KeyboardInterrupt: 